In [5]:
from skyfield.api import Topos, load, EarthSatellite
import requests

# Space-Track login
username = 'mathuppk@gmail.com'
password = 'JimsSatellites10!'

# TLE data of your satellite
TLE_Jim1 = [
    "1 25544U 98067A   21274.46578241  .00001480  00000+0  34227-3 0  9990",
    "2 00004  104.0000  232.7000  0012306  100.0000  103.0000  14.99449996"
]

# Space-Track API access
url = "https://www.space-track.org/ajaxauth/login"
query = {
    "query": "https://www.space-track.org/basicspacedata/query/class/tle_latest/ORDINAL/1/NORAD_CAT_ID/25544--44109/orderby/NORAD_CAT_ID/format/tle",
    "identity": username,
    "password": password,
}

# TLE data from Space-Track
response = requests.post(url, data=query)
if response.status_code == 200: 
    tle_data = response.text.split("\r\n") # For some reason the TLE data is weridly formatted for some, so this is necessary
else:
    print("Failed to access TLE data.")
    tle_data = []

# Divide TLE data into individual TLEs
TLECompile = [TLE_Jim1] + [tle_data[i:i + 2] for i in range(0, len(tle_data), 2) if len(tle_data[i:i + 2]) == 2]

# Solution array to store Skyfield satellite objects
satellites = []

for tle in TLECompile:
    if len(tle) == 2:  # Verify that the TLE has two lines, for some this is a problem
        line1, line2 = tle
        satellite = EarthSatellite(line1, line2)
        satellites.append(satellite)

# Identify Jim-1 satellite and others
Jim1 = satellites[0]
OtherSat = satellites[1:]

ts = load.timescale() # Time scale

# Simulation. It will iterate through each minute for a given time (e.g., 3 hours = 180 minutes)
for minute in range(180):
    t = ts.utc(2023, 10, 13, 0, minute) # Time and date
    Jum1Pos = Jim1.at(t).position.km # Obtains position of Jim-1 satellite
    # This loop will iterate through other satellites to find close approaches (within 1 km)
    for sat in OtherSat:
        OtherSatPos = sat.at(t).position.km # Gets position of the other satellite
        distance = ((Jum1Pos - OtherSatPos)**2).sum()**0.5  # Determine relative distance between satellites
        # Check for close approach (threshold, e.g., 1 km)
        if distance < 1.0:
            print(f"Close approach with {sat.name} at distance {distance:.2f} km at time {t.utc_iso()}.")
